# Weather Data Scraping
#### Fire Atlas Dataset
Andrew McDonald // CSE 847 // April 2021

In [1]:
import requests
import urllib
import json
from datetime import datetime

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
ignitions = gpd.read_file("2016_wildfires/usa_2016_ignitions_stations.shp")
ignitions

,fire_ID,latitude_x,longitude_,size,perimeter,start_date,start_DOY,end_date,end_DOY,duration,...,mindate,maxdate,latitude_y,name_y,datacovera,id,elevationU,longitud_1,distance_f,geometry
0,1,20.8438,-156.4130,0.86,4.63,2016-10-26,300,2016-11-02,307,8,...,1949-10-01,2016-10-01,20.85710,"PUUNENE 396, HI US",0.9875,GHCND:USC00518543,METERS,-156.43130,0.022623,POINT (-156.41374 20.84379)
1,2,20.8396,-156.4180,1.50,6.48,2016-10-28,302,2016-11-02,307,6,...,1949-10-01,2016-10-01,20.85710,"PUUNENE 396, HI US",0.9875,GHCND:USC00518543,METERS,-156.43130,0.021980,POINT (-156.41833 20.83962)
2,3,20.8105,-156.4370,0.43,2.78,2016-11-29,334,2016-11-29,334,1,...,1905-01-01,2016-10-01,20.79120,"KIHEI 311, HI US",0.6364,GHCND:USC00514489,METERS,-156.44210,0.019962,POINT (-156.43708 20.81045)
3,4,20.6271,-156.2470,18.65,30.56,2016-02-16,47,2016-02-20,51,5,...,2013-01-01,2017-03-01,20.68420,"KAUPO GAP HAWAII, HI US",1.0000,GHCND:USR0000HKAU,METERS,-156.15190,0.110925,POINT (-156.24786 20.62712)
4,5,19.8063,-155.8950,7.72,12.96,2016-03-20,80,2016-03-25,85,6,...,2003-05-01,2021-03-01,19.79500,"PUU WAAWAA HAWAII, HI US",0.9487,GHCND:USR0000HPUW,METERS,-155.84530,0.050968,POINT (-155.89512 19.80629)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11519,56789,40.8521,-78.1691,0.43,2.78,2016-04-14,105,2016-04-14,105,1,...,1998-04-01,2021-03-01,40.87291,"PHILIPSBURG 2 S, PA US",0.9927,GHCND:USC00366921,METERS,-78.21615,0.051447,POINT (-78.16909 40.85211)
11520,56790,40.4188,-76.7056,0.64,3.70,2015-11-14,318,2015-11-14,318,1,...,1998-05-01,2021-03-01,40.46093,"DEHART DAM, PA US",0.8872,GHCND:USC00362071,METERS,-76.74870,0.060271,POINT (-76.70562 40.41878)
11521,56791,40.0271,-74.4895,0.86,4.63,2016-06-26,178,2016-07-04,186,9,...,2013-08-01,2020-06-01,40.06628,"JACKSON TWP 2.2 S, NJ US",0.5904,GHCND:US1NJOC0053,METERS,-74.35698,0.138191,POINT (-74.48954 40.02711)
11522,56792,40.0063,-74.4994,3.00,9.26,2016-06-13,165,2016-06-19,171,7,...,2013-08-01,2020-06-01,40.06628,"JACKSON TWP 2.2 S, NJ US",0.5904,GHCND:US1NJOC0053,METERS,-74.35698,0.154536,POINT (-74.49944 40.00628)


## 1. Explore Data Types

In [3]:
api_token = "syKpWfFaNGhSsksXrJhcrwQspIYihUqW"
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/"
headers = {"token": api_token}

In [4]:
resource = "datatypes?"
query = {
    "datasetid": "GSOM",
    "startdate": "2016-01-01",
    "enddate": "2016-01-01",
    "limit": "1000"
}
query_str = urllib.parse.urlencode(query, doseq=False)
endpoint = base_url + resource + query_str
endpoint

'https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?datasetid=GSOM&startdate=2016-01-01&enddate=2016-01-01&limit=1000'

In [5]:
response = requests.get(endpoint, headers=headers)
response

<Response [200]>

In [6]:
data_types = json.loads(response.text)
print(data_types["metadata"])
print(data_types["results"][0])

{'resultset': {'offset': 1, 'count': 89, 'limit': 1000}}
{'mindate': '1982-01-01', 'maxdate': '2021-03-01', 'name': 'Average wind speed', 'datacoverage': 1, 'id': 'AWND'}


In [7]:
data_types_df = pd.DataFrame(data_types["results"])
data_types_df.to_csv("2016_wildfires/weather_data_types.csv")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(data_types_df)

,mindate,maxdate,name,datacoverage,id
0,1982-01-01,2021-03-01,Average wind speed,1.0000,AWND
1,1763-01-01,2021-03-01,Cooling Degree Days Season to Date,1.0000,CDSD
2,1763-01-01,2021-03-01,Cooling Degree Days,1.0000,CLDD
3,1781-01-01,2021-03-01,Number of days with greater than or equal to 0...,1.0000,DP01
4,1781-01-01,2021-03-01,Number of days with greater than or equal to 1...,1.0000,DP10
5,1860-01-01,2021-03-01,Number days with snow depth > 1 inch(25.4mm) f...,1.0000,DSND
6,1840-05-01,2021-03-01,Number days with snow depth > 1 inch.,1.0000,DSNW
7,1763-01-01,2021-03-01,Number days with minimum temperature less than...,1.0000,DT00
8,1763-01-01,2021-03-01,Number days with minimum temperature less than...,1.0000,DT32
9,1763-01-01,2021-03-01,Number days with maximum temperature < 32 F.,1.0000,DX32


## 2. Explore Station Locations


In [8]:
resource = "locations?"
query = {
    "datasetid": "GSOM",
    "startdate": "2016-01-01",
    "enddate": "2016-01-01",
    "locationcategoryid": "ZIP",
    "limit": "1000"
}
query_str = urllib.parse.urlencode(query, doseq=False)
endpoint = base_url + resource + query_str
display(endpoint)

response = requests.get(endpoint, headers=headers)
display(response)

locations = json.loads(response.text)
display(locations["metadata"])
display(locations["results"][0])

locations_df = pd.DataFrame(locations["results"])
display(locations_df)

'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?datasetid=GSOM&startdate=2016-01-01&enddate=2016-01-01&locationcategoryid=ZIP&limit=1000'

<Response [200]>

{'resultset': {'offset': 1, 'count': 13116, 'limit': 1000}}

{'mindate': '1900-11-01',
 'maxdate': '2021-03-01',
 'name': 'Yukon Flats Nat Wildlife, AK 00002',
 'datacoverage': 1,
 'id': 'ZIP:00002'}

,mindate,maxdate,name,datacoverage,id
0,1900-11-01,2021-03-01,"Yukon Flats Nat Wildlife, AK 00002",1.0000,ZIP:00002
1,1927-07-01,2021-03-01,"Alaska Peninsula NWR, AK 00003",1.0000,ZIP:00003
2,1936-07-01,2021-03-01,"W Kenai Peninsula Boroug, AK 00004",1.0000,ZIP:00004
3,1940-04-01,2021-03-01,"N Lake and Peninsula Bor, AK 00005",1.0000,ZIP:00005
4,1918-01-01,2021-03-01,"Matanuska-Sustina Bor, AK 00006",1.0000,ZIP:00006
...,...,...,...,...,...
995,1909-03-01,2021-03-01,"Northville, NY 12134",1.0000,ZIP:12134
996,2013-08-01,2021-03-01,"Pattersonville, NY 12137",0.9892,ZIP:12137
997,1899-03-01,2017-02-01,"Richmondville, NY 12149",0.9349,ZIP:12149
998,2011-01-01,2021-03-01,"Schoharie, NY 12157",0.9840,ZIP:12157


In [9]:
resource = "stations?"
query = {
    "datasetid": "GSOM",
    "startdate": "2016-01-01",
    "enddate": "2016-01-01",
    "limit": "1000"
}
query_str = urllib.parse.urlencode(query, doseq=False)
endpoint = base_url + resource + query_str
display(endpoint)

response = requests.get(endpoint, headers=headers)
display(response)

stations = json.loads(response.text)
display(stations["metadata"])
display(stations["results"][0])

stations_df = pd.DataFrame(stations["results"])
display(stations_df)

'https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=GSOM&startdate=2016-01-01&enddate=2016-01-01&limit=1000'

<Response [200]>

{'resultset': {'offset': 1, 'count': 35678, 'limit': 1000}}

{'elevation': 34,
 'mindate': '1944-04-01',
 'maxdate': '2021-03-01',
 'latitude': 25.333,
 'name': 'SHARJAH INTER. AIRP, AE',
 'datacoverage': 0.4643,
 'id': 'GHCND:AE000041196',
 'elevationUnit': 'METERS',
 'longitude': 55.517}

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
0,34.0,1944-04-01,2021-03-01,25.3330,"SHARJAH INTER. AIRP, AE",0.4643,GHCND:AE000041196,METERS,55.5170
1,10.4,1983-01-01,2021-03-01,25.2550,"DUBAI INTERNATIONAL, AE",0.6362,GHCND:AEM00041194,METERS,55.3640
2,26.8,1983-06-01,2021-03-01,24.4330,"ABU DHABI INTERNATIONAL, AE",0.6013,GHCND:AEM00041217,METERS,54.6510
3,264.9,1994-04-01,2021-03-01,24.2620,"AL AIN INTERNATIONAL, AE",0.5123,GHCND:AEM00041218,METERS,55.6090
4,977.2,1979-07-01,2020-12-01,34.2100,"HERAT, AF",0.0080,GHCND:AFM00040938,METERS,62.2280
...,...,...,...,...,...,...,...,...,...
995,315.0,2003-07-01,2020-12-01,-33.2064,"BEAUMONT, AS",0.3475,GHCND:ASN00010816,METERS,117.6994
996,300.0,1969-09-01,2020-12-01,-32.2344,"GLENMORE, AS",0.9334,GHCND:ASN00010823,METERS,117.6444
997,330.0,1953-12-01,2021-02-01,-33.4836,"GLENROSE, AS",0.2491,GHCND:ASN00010831,METERS,117.6100
998,400.0,1970-10-01,2017-11-01,-32.8192,"MINNI DOWNS, AS",0.8251,GHCND:ASN00010851,METERS,119.2936


## 3. Explore Actual Data


In [10]:
unique_stations = ignitions["id"].unique()
len(unique_stations)

2837

In [11]:
resource = "data?"
query = {
    "stationid": f"{unique_stations[0]}",
    "datasetid": "GSOM",
    "startdate": "2016-01-01",
    "enddate": "2016-12-31",
    "units": "metric",
    "limit": "1000"
}
query_str = urllib.parse.urlencode(query, doseq=False)
endpoint = base_url + resource + query_str
display(endpoint)

response = requests.get(endpoint, headers=headers)
display(response)

data = json.loads(response.text)
display(data["metadata"])
display(data["results"][:2])

display(pd.unique([d["datatype"] for d in data["results"]]))
display(pd.unique([d["date"] for d in data["results"]]))
display(pd.unique([d["station"] for d in data["results"]]))

'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?stationid=GHCND%3AUSC00518543&datasetid=GSOM&startdate=2016-01-01&enddate=2016-12-31&units=metric&limit=1000'

<Response [200]>

{'resultset': {'offset': 1, 'count': 60, 'limit': 1000}}

[{'date': '2016-01-01T00:00:00',
  'datatype': 'DP01',
  'station': 'GHCND:USC00518543',
  'attributes': ',7',
  'value': 4},
 {'date': '2016-01-01T00:00:00',
  'datatype': 'DP10',
  'station': 'GHCND:USC00518543',
  'attributes': ',7',
  'value': 2}]

array(['DP01', 'DP10', 'DP1X', 'DYXP', 'EMXP', 'PRCP'], dtype=object)

array(['2016-01-01T00:00:00', '2016-02-01T00:00:00',
       '2016-03-01T00:00:00', '2016-04-01T00:00:00',
       '2016-05-01T00:00:00', '2016-06-01T00:00:00',
       '2016-07-01T00:00:00', '2016-08-01T00:00:00',
       '2016-09-01T00:00:00', '2016-10-01T00:00:00'], dtype=object)

array(['GHCND:USC00518543'], dtype=object)

## 4. Obtain All 2016 Weather Data

In [12]:
# weather = []
# resource = "data?"
# query = {
#     "datasetid": "GSOM",
#     "startdate": "2016-01-01",
#     "enddate": "2016-12-31",
#     "units": "metric",
#     "limit": "1000"
# }
# 
# for i, station in enumerate(unique_stations):
#     query["stationid"] = station
#     query_str = urllib.parse.urlencode(query, doseq=False)
#     endpoint = base_url + resource + query_str
#     response = requests.get(endpoint, headers=headers)
#     results = json.loads(response.text)
#     if "results" in results:
#         weather.append(results["results"])
#     if i % 100 == 0:
#         print(f"Finished station {i}")

In [13]:
# rows = []
# for station_data in weather:
#     # datetime.fromisoformat(entry["date"]).month
#     i = 0
#     row = {"date": None}
#     for i, variable in enumerate(station_data):
#         if row["date"] != variable["date"]:
#             if i > 0:
#                 rows.append(row)
#             row = {"date": variable["date"], "station": variable["station"]}
#         row[variable["datatype"]] = variable["value"]

In [14]:
# weather_df = pd.DataFrame(rows)
# weather_df["month"] = pd.DatetimeIndex(weather_df["date"]).month
# weather_df.to_csv("2016_wildfires/2016_weather.csv")
# weather_df

## 5. Join Weather Data to Wildfire Data

In [15]:
# ignitions["month"] = pd.DatetimeIndex(ignitions["start_date"]).month
# ignitions

In [16]:
# ignitions_weather = pd.merge(ignitions, weather_df,
#                              left_on=["id", "month"],
#                              right_on=["station", "month"])
# ignitions_weather.to_file("2016_wildfires/usa_2016_ignitions_weather.shp")
# with pd.option_context('display.max_columns', None):  # more options can be specified also
#     display(ignitions_weather)